# Object Detection on Drones

In [11]:
from PIL import Image
from random import randint
import matplotlib.pyplot as plt
%matplotlib inline  
import cv2 as cv
import os
import os.path
import splitfolders
from numpy import asarray
import pandas as pd
import numpy as np

# Squishing Images

For the first part of the Data Preprocessing I need to get all the images to be the same size. This will squish the images to 960x540 pixels. Squishing all of my images and putting them into one folder to save all that data to then have the drones coppied on to it.

Setting what directory to get the images to squish and where to send them.

In [ ]:
ims = r'/home/alec/Desktop/FlatIron/phase_4/blog4/test/'
out = r'/home/alec/Desktop/FlatIron/phase_4/blog4/bigger/'

This Will create the directory to store the squished directory if it doesnt already exist. Will also go through and squish the data if the Directory doesn't exsist.

In [ ]:
if not os.path.isdir(out):
        os.mkdir(out)
for file in os.listdir(ims):
    f_img = ims + '/' + file
    out_img = out + '/' + file
    img = Image.open(f_img)
    img = img.resize((960,540))
    img.save(out_img)

## Create The Labels for images

In [ ]:
def create_kitti(background_path, drone_path,itteration):
    with open(f'{kitti}{background_path[:-4]}_{drone_path[:-4]}_{itteration}.txt', 'w') as f:
        truncated, occluded, alpha = '0.00', '0', '0.00'
        dim_x, dim_y, dim_z, loc_x, loc_y, loc_z, rotation_y = '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00'
        annotation_row = f"drone {truncated} {occluded} {alpha} {x_min} {y_min} {x_max} " \
                         f"{y_max} {dim_x} {dim_y} {dim_z} {loc_x} {loc_y} {loc_z} {rotation_y}"
        f.write(f'{annotation_row}\n')

## Creating the Mask for the Drones 

This will take in the image of the drone and mask over just the drone so when we paste the image into the background image with the mask it will not have the white background in the images.

This will take in the testing image for the background and also the drone image to be placed on top of the image.

In [ ]:
background_image_path = 'test2.jpg'
drone_path = 'drone_020.jpg'
print(drone_path[:-4])

This creates the mask

In [ ]:
pix_thresh = 245
foreground_image = Image.open(drone_path)
foreground_max_x = foreground_image.size[0]
foreground_max_y = foreground_image.size[1]
mask = Image.new('1', (foreground_max_x, foreground_max_y))
for x in range(foreground_max_x):
    for y in range(foreground_max_y):
        if (sum(foreground_image.getpixel((x,y))) / 3) > pix_thresh:
            mask.putpixel((x,y), 0)
        else:
            mask.putpixel((x,y), 1)
new_x = 65
new_y = 65
foreground_image = foreground_image.resize((new_x, new_y))
mask = mask.resize((new_x, new_y))

In [ ]:
# mask.show()

Creates a test to see that the bounding box is being put in the right place. before generating the images and saving them

In [ ]:
background_image = Image.open(background_image_path)
background_max_x = background_image.size[0]
background_max_y = background_image.size[1]
copy_paste_x = randint(0, background_max_x - new_x)
copy_paste_y = randint(0, background_max_y - new_y)
copy_paste_image = background_image.paste(foreground_image, (copy_paste_x, copy_paste_y), mask)

data = asarray(background_image)
x_max = (copy_paste_x + 70) 
y_max = (copy_paste_y + 70)
x_min = (copy_paste_x)
y_min = (copy_paste_y)
start = ((x_min), (y_min))
end = ((x_max), (y_max))
color = (52, 235, 225)

cv.rectangle(data, start, end, color, 2)
from matplotlib import pyplot as plt
plt.imshow(data, interpolation='nearest')
plt.show()

# background_image.show()

In [ ]:
# cv.rectangle(data, start, end, color, 2)
# from matplotlib import pyplot as plt
# plt.imshow(data, interpolation='nearest')
# plt.show()

Generating all the images: 5 images per-background per drone.

In [ ]:
old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/bigger/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/D_Alec_Flatiron_Drone/D_1/train/images/'
kitti = r'/home/alec/Desktop/FlatIron/phase_4/blog4/D_Alec_Flatiron_Drone/D_1/train/kitti-labels/'
#if not os.path.isdir(new):
#       os.mkdir(new)
i=1
while i <= 5:
    for file in os.listdir(old):
        f_img = f'{old}/{file}'
        drone_img = f'{new}{file[:-4]}_{drone_path[:-4]}_{i}.jpg'
        background_image = Image.open(f_img)
        background_max_x = background_image.size[0]
        background_max_y = background_image.size[1]
        copy_paste_x = randint(0, background_max_x - new_x)
        copy_paste_y = randint(0, background_max_y - new_y)
        copy_paste_image = background_image.paste(foreground_image, (copy_paste_x, copy_paste_y), mask)
        x_max = (copy_paste_x + 70) 
        y_max = (copy_paste_y + 70)
        x_min = (copy_paste_x)
        y_min = (copy_paste_y)
        create_kitti(file, drone_path,i)
        background_image.save(drone_img)
    print(i)
    i= i +1

Testing to make sure the its the same for the .jpg and the .txt file

In [ ]:
print(drone_img)
print(f'{kitti}17104543062_7f23202df6_c_{drone_path[:-4]}_5.txt')

# Flipping and rotating images

test_image = 'image_fav.jpg'
image = cv.imread('image_fav.jpg') 
img = cv.rotate(image, cv.ROTATE_90_CLOCKWISE)

Rotating Images 90 degrees clockwise

old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/Drones_background/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/Images_rotated/'
if not os.path.isdir(new):
      os.mkdir(new)
for file in os.listdir(old):
    f_img = old + '/' + file
    drone_img = new + '/'+ 'ROTATE_90_CLOCKWISE' + file
    image = cv.imread(f_img) 
    img = cv.rotate(image, cv.ROTATE_90_CLOCKWISE)
    cv.imwrite(drone_img,img)

Rotating Images 180 degrees

old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/Drones_background/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/Images_rotated/'
#if not os.path.isdir(new):
#       os.mkdir(new)
for file in os.listdir(old):
    f_img = old + '/' + file
    drone_img = new + '/'+ 'ROTATE_180' + file
    image = cv.imread(f_img) 
    img = cv.rotate(image, cv.ROTATE_180)
    cv.imwrite(drone_img,img)

Rotating images 90 degrees counter clockwise

old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/Drones_background/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/Images_rotated/'
#if not os.path.isdir(new):
#       os.mkdir(new)
for file in os.listdir(old):
    f_img = old + '/' + file
    drone_img = new + '/'+ 'ROTATE_90_COUNTERCLOCKWISE' + file
    image = cv.imread(f_img) 
    img = cv.rotate(image, cv.ROTATE_90_COUNTERCLOCKWISE)
    cv.imwrite(drone_img,img)

Flippng the data left to right

old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/Drones_background/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/Images_rotated/'
#if not os.path.isdir(new):
#       os.mkdir(new)
for file in os.listdir(old):
    f_img = old + '/' + file
    drone_img = new + '/'+ 'FLIP_LEFT_RIGHT' + file
    image = Image.open(f_img) 
    img = image.transpose(Image.FLIP_LEFT_RIGHT)
    img.save(drone_img)

Flipping the data from top to bottom

old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/Drones_background/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/Images_rotated/'
#if not os.path.isdir(new):
#       os.mkdir(new)
for file in os.listdir(old):
    f_img = old + '/' + file
    drone_img = new + '/'+ 'FLIP_TOP_BOTTOM' + file
    image = Image.open(f_img) 
    img = image.transpose(Image.FLIP_TOP_BOTTOM)
    img.save(drone_img)

### Creating the No Drone Dataset
altered the project path and Won't Need a no drone class

Next I need to make sure that I dont over fit my data on images with drones so I got more images and will flip and rotate all of these images

Rotating 90 degrees

old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/no_drone/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/no_drone_rotated/'
#if not os.path.isdir(new):
       os.mkdir(new)
for file in os.listdir(old):
    f_img = old + '/' + file
    drone_img = new + '/'+ 'ROTATE_90_CLOCKWISE' + file
    image = cv.imread(f_img) 
    img = cv.rotate(image, cv.ROTATE_90_CLOCKWISE)
    cv.imwrite(drone_img,img)

Rotating 180 degrees

old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/no_drone/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/no_drone_rotated/'
if not os.path.isdir(new):
      os.mkdir(new)
for file in os.listdir(old):
    f_img = old + '/' + file
    drone_img = new + '/'+ 'ROTATE_180' + file
    image = cv.imread(f_img) 
    img = cv.rotate(image, cv.ROTATE_180)
    cv.imwrite(drone_img,img)

Rotating images 90 degrees counter-clockwise

old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/no_drone/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/no_drone_rotated/'
#if not os.path.isdir(new):
<!-- #       os.mkdir(new) -->
for file in os.listdir(old):
    f_img = old + '/' + file
    drone_img = new + '/'+ 'ROTATE_90_COUNTERCLOCKWISE' + file
    image = cv.imread(f_img) 
    img = cv.rotate(image, cv.ROTATE_90_COUNTERCLOCKWISE)
    cv.imwrite(drone_img,img)

Flipping the images without drones from top to bottom

old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/no_drone/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/no_drone_rotated/'
if not os.path.isdir(new):
      os.mkdir(new)
for file in os.listdir(old):
    f_img = old + '/' + file
    drone_img = new + '/'+ 'FLIP_LEFT_RIGHT' + file
    image = Image.open(f_img) 
    img = image.transpose(Image.FLIP_LEFT_RIGHT)
    img.save(drone_img)

Flipping the images with drones from left to right

old = r'/home/alec/Desktop/FlatIron/phase_4/blog4/no_drone/'
new = r'/home/alec/Desktop/FlatIron/phase_4/blog4/no_drone_rotated/'
if not os.path.isdir(new):
      os.mkdir(new)
for file in os.listdir(old):
    f_img = old + '/' + file
    drone_img = new + '/'+ 'FLIP_TOP_BOTTOM' + file
    image = Image.open(f_img) 
    img = image.transpose(Image.FLIP_TOP_BOTTOM)
    img.save(drone_img)

# Splitting data

In [ ]:
splitfolders.ratio("both", output="Data_Split",
 seed=42, ratio=(.64, .16, .2), group_prefix=None, move=True)

# Evaluating the Model

To Evaluate the model the Score that means the most for this model is the IOU score

In [17]:
def get_iou(boxA: list, boxB: list) -> float:
    '''Calculate the IOU of two boxes'''
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [18]:
df = pd.read_csv('infer_results.csv')

In [19]:
test_eval = df[['bbox_xmin', 'bbox_ymin', 'bbox_xmax', 'bbox_ymax', 'external_id', 'predicted_label' ]]
test_eval.tail()

,bbox_xmin,bbox_ymin,bbox_xmax,bbox_ymax,external_id,predicted_label
17794,704,261,774,329,0000631_drone_013_1.jpg,P_1
17795,261,133,330,201,4100246_drone_007_2.jpg,P_1
17796,233,306,302,375,3001555_drone_011_2.jpg,P_1
17797,878,98,949,169,32446880977_a94ed562b5_c_drone_012_4.jpg,P_1
17798,278,402,348,471,6000203_drone_013_3.jpg,P_1


In [20]:
predicted = np.array([0,363,32,429])
actual = np.array([278,348,402,471])
print(actual)

[278 348 402 471]


In [21]:
for i in test_eval.index:
    p_x_min = test_eval.loc[i, 'bbox_xmin']
    p_x_max = test_eval.loc[i, 'bbox_xmax']
    p_y_min = test_eval.loc[i, 'bbox_ymin']
    p_y_max = test_eval.loc[i, 'bbox_ymax']
    predicted = [p_x_min, p_x_max, p_y_min, p_y_max]
    e_id = test_eval.loc[i, 'external_id']
    f = open(f'/home/alec/Desktop/FlatIron/phase_4/blog4/D_Alec_Flatiron_Drone/D_1/train/kitti-labels/{e_id[:-4]}.txt', 'r')
    test = f.read()
    test_list = test.split()
    a_label = test_list[0]
    a_x_min = test_list[4]
    a_y_min = test_list[5]
    a_x_max = test_list[6]
    a_y_max = test_list[7]
    actual = [int(a_x_min), int(a_x_max), int(a_y_min), int(a_y_max)]
    iou = get_iou(actual, predicted)
print(iou)

ZeroDivisionError: float division by zero

In [22]:
actual = np.array([int(a_x_min), int(a_x_max), int(a_y_min), int(a_y_max)])
print(actual)
print(predicted)

[285 355 285 355]
[284, 355, 284, 353]


In [ ]:
f = open('/home/alec/Desktop/FlatIron/phase_4/blog4/D_Alec_Flatiron_Drone/D_1/train/kitti-labels/0000047_drone_001_1.txt', 'r')

In [ ]:
test = f.read()
test_list = test.split()
a_label = test_list[0]
a_x_min = test_list[4]
a_y_min = test_list[5]
a_x_max = test_list[6]
a_y_max = test_list[7]
print(f'a_x_min: {a_x_min}, a_x_max: {a_x_max}, a_y_min: {a_y_min}, a_y_max: {a_y_max}')

In [ ]:
bb_intersection_over_union(actual, predicted)

In [ ]:
print(actual)

In [ ]:
print(predicted)

# Model Stats

In [29]:
model_stats = pd.read_csv('infer_results.csv')

In [30]:
model_stats

,Unnamed: 0,test_id,dab_id,bbox_xmin,bbox_ymin,bbox_xmax,bbox_ymax,predicted_label,external_id,confidence,...,r_means,b_stds,g_stds,r_stds,roi_b_means,roi_g_means,roi_r_means,roi_b_stds,roi_g_stds,roi_r_stds
0,0,1000,1,695,158,765,228,P_1,0000087_drone_001_2.jpg,0.945687,...,118.712265,77.448364,76.141690,75.322190,78.973880,74.797960,78.061226,43.316334,41.535450,37.858770
1,1,1000,1,452,156,521,225,P_1,2000031_drone_015_2.jpg,0.924940,...,130.148800,79.320470,54.743008,43.092830,197.803620,186.920180,181.368620,51.037125,50.253677,50.282814
2,2,1000,1,599,407,669,476,P_1,0100091_drone_017_3.jpg,0.947662,...,130.682310,69.839600,70.992640,68.209360,86.291100,83.159004,93.043274,38.484547,33.891327,36.104600
3,3,1000,1,668,196,739,265,P_1,6000080_drone_013_2.jpg,0.923659,...,119.398080,68.162400,63.297638,63.577420,169.020610,180.046540,190.254750,63.213787,61.141857,60.083523
4,4,1000,1,87,340,156,409,P_1,5000131_drone_007_4.jpg,0.941192,...,125.689445,76.538720,69.860590,76.490580,88.790800,98.316950,107.111320,50.223564,49.712360,50.051003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17794,17794,1000,1,704,261,774,329,P_1,0000631_drone_013_1.jpg,0.946999,...,122.695496,59.283325,56.967907,55.326794,113.524370,115.475000,113.653150,48.746418,56.106556,61.844910
17795,17795,1000,1,261,133,330,201,P_1,4100246_drone_007_2.jpg,0.938510,...,125.926790,77.981870,63.614990,57.862045,88.438410,120.493180,130.882350,43.454506,40.201940,41.940563
17796,17796,1000,1,233,306,302,375,P_1,3001555_drone_011_2.jpg,0.945693,...,113.773094,80.840645,71.006890,69.186960,63.873768,78.722950,78.711620,60.789173,55.016582,54.999570
17797,17797,1000,1,878,98,949,169,P_1,32446880977_a94ed562b5_c_drone_012_4.jpg,0.936183,...,137.173280,75.231310,64.356490,63.259598,93.731400,69.528070,69.109310,35.864487,37.114227,40.077736
